In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import classification_report , confusion_matrix , accuracy_score
from sklearn.model_selection import train_test_split

import cv2
#from google.colab.patches import cv2_imshow
from PIL import Image 
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Input, Dense,Conv2D , MaxPooling2D, Flatten,BatchNormalization,Dropout
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow_hub as hub 

## Data Collection

In [ ]:
train_path = "/kaggle/input/brain-tumor-classification-mri/Training"
test_path = "/kaggle/input/brain-tumor-classification-mri/Testing"

gl_tr = "/kaggle/input/brain-tumor-classification-mri/Training/glioma_tumor"
men_tr = "/kaggle/input/brain-tumor-classification-mri/Training/meningioma_tumor"
no_tr = "/kaggle/input/brain-tumor-classification-mri/Training/no_tumor"
pit_tr = "/kaggle/input/brain-tumor-classification-mri/Training/pituitary_tumor"


gl_ts = "/kaggle/input/brain-tumor-classification-mri/Testing/glioma_tumor"
men_ts = "/kaggle/input/brain-tumor-classification-mri/Testing/meningioma_tumor"
no_ts = "/kaggle/input/brain-tumor-classification-mri/Testing/no_tumor"
pit_ts = "/kaggle/input/brain-tumor-classification-mri/Testing/pituitary_tumor"

gl_tr = os.listdir(gl_tr)
men_tr = os.listdir(men_tr)
no_tr = os.listdir(no_tr)
pit_tr = os.listdir(pit_tr)

gl_ts = os.listdir(gl_ts)
men_ts = os.listdir(men_ts)
no_ts = os.listdir(no_ts)
pit_ts = os.listdir(pit_ts)

In [ ]:
print(len(gl_tr))
print(len(men_tr))
print(len(no_tr))
print(len(pit_tr))

In [ ]:
print(len(gl_ts))
print(len(men_ts))
print(len(no_ts))
print(len(pit_ts))

## Class Labels

In [ ]:
gl_tr_label = [0]*len(gl_tr)
men_tr_label = [1]*len(men_tr)
no_tr_label = [2]*len(no_tr)
pit_tr_label = [3]*len(pit_tr)

gl_ts_label = [0]*len(gl_ts)
men_ts_label = [1]*len(men_ts)
no_ts_label = [2]*len(no_ts)
pit_ts_label = [3]*len(pit_ts)

In [ ]:
train_label = gl_tr_label + men_tr_label + no_tr_label + pit_tr_label
test_label = gl_ts_label + men_ts_label + no_ts_label + pit_ts_label

In [ ]:
print(len(train_label))
print(len(test_label))

In [ ]:
train_label[:4]

In [ ]:
test_label[-5:]

In [ ]:
class_labels = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

In [ ]:
label = train_label + test_label
len(label)

In [ ]:
y = np.array(label)

In [ ]:
y

## Images Data

In [ ]:
# gl_tr = "/kaggle/input/brain-tumor-classification-mri/Training/glioma_tumor"
# men_tr = "/kaggle/input/brain-tumor-classification-mri/Training/meningioma_tumor"
# no_tr = "/kaggle/input/brain-tumor-classification-mri/Training/no_tumor"
# pit_tr = "/kaggle/input/brain-tumor-classification-mri/Training/pituitary_tumor"


# gl_ts = "/kaggle/input/brain-tumor-classification-mri/Testing/glioma_tumor"
# men_ts = "/kaggle/input/brain-tumor-classification-mri/Testing/meningioma_tumor"
# no_ts = "/kaggle/input/brain-tumor-classification-mri/Testing/no_tumor"
# pit_ts = "/kaggle/input/brain-tumor-classification-mri/Testing/pituitary_tumor"

data = []

## For Training Data
for img_file in gl_tr:
    image = Image.open("/kaggle/input/brain-tumor-classification-mri/Training/glioma_tumor/"+img_file)
    image = image.resize((224,224))
    image = np.array(image)
    data.append(image)
    
for img_file in men_tr:
    image = Image.open("/kaggle/input/brain-tumor-classification-mri/Training/meningioma_tumor/"+img_file)
    image = image.resize((224,224))
    image = np.array(image)
    data.append(image)
    
for img_file in no_tr:
    image = Image.open("/kaggle/input/brain-tumor-classification-mri/Training/no_tumor/"+img_file)
    image = image.resize((224,224))
    image = np.array(image)
    data.append(image)
    
for img_file in pit_tr:
    image = Image.open("/kaggle/input/brain-tumor-classification-mri/Training/pituitary_tumor/"+img_file)
    image = image.resize((224,224))
    image = np.array(image)
    data.append(image)
    
    
## For Testing Data
    
for img_file in gl_ts:
    image = Image.open("/kaggle/input/brain-tumor-classification-mri/Testing/glioma_tumor/"+img_file)
    image = image.resize((224,224))
    image = np.array(image)
    data.append(image)
    
for img_file in men_ts:
    image = Image.open("/kaggle/input/brain-tumor-classification-mri/Testing/meningioma_tumor/"+img_file)
    image = image.resize((224,224))
    image = np.array(image)
    data.append(image)
    
for img_file in no_ts:
    image = Image.open("/kaggle/input/brain-tumor-classification-mri/Testing/no_tumor/"+img_file)
    image = image.resize((224,224))
    image = np.array(image)
    data.append(image)
    
for img_file in pit_ts:
    image = Image.open("/kaggle/input/brain-tumor-classification-mri/Testing/pituitary_tumor/"+img_file)
    image = image.resize((224,224))
    image = np.array(image)
    data.append(image)

In [ ]:
x = np.array(data)

In [ ]:
x[0]

In [ ]:
print(x.shape)
print(y.shape)

## Split The Data

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.10,shuffle=True)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

## Train Data Image Visualization

In [ ]:
plt.figure(figsize=(16,20))
for i in range(0,20):
    plt.subplot(5,4,i+1)
    plt.imshow(x_train[i])
    plt.title(f"Label:{class_labels[y_train[i]]}")
    plt.axis("off")

## Scale The Data

In [ ]:
x_train_scaled = x_train/255
x_test_scaled = x_test/255

# Using MobileNet V2 Pre-trained Model

In [ ]:
# MobileNet V2 Classification 100%  224x224
path1 ="https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/classification/5"

path="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
pretrained_model = hub.KerasLayer(path1, input_shape=(224,224,3), trainable=False)

In [ ]:
num_class = 4
pre_model = Sequential()
pre_model.add(pretrained_model)
pre_model.add(Dense(units=num_class,activation="softmax"))

pre_model.summary()

In [ ]:
pre_model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
history_1 = pre_model.fit(x_train_scaled, y_train ,
          batch_size=32, epochs=15, validation_data = (x_test_scaled,y_test))

## Model Evaluation on Test & Train Data

In [ ]:
loss, acc = pre_model.evaluate(x_train_scaled,y_train)
print("Loss on Train Data:",loss)
print("Accuracy on Train Data:",acc)
print()
val_loss, val_acc = pre_model.evaluate(x_test_scaled,y_test)
print("Loss on Test Data:",val_loss)
print("Accuracy on Test Data:",val_acc)

In [ ]:
acc = history_1.history["accuracy"]
val_acc = history_1.history["val_accuracy"]

loss = history_1.history["loss"]
val_loss = history_1.history["val_loss"]

## Training and Validation Accuracy Plot

In [ ]:
EPOCHS=15
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.plot(range(EPOCHS),acc, label="Training Accuracy")
plt.plot(range(EPOCHS),val_acc, label="Validation Accuracy")
plt.legend(loc="lower right")
plt.title("Training and Validation Accuracy")

#plt.figure(figsize=(6,6))
plt.subplot(1,2,2)
plt.plot(range(EPOCHS),loss, label="Training Loss")
plt.plot(range(EPOCHS),val_loss, label="Validation Loss")
plt.legend(loc="lower right")
plt.title("Training and Validation Loss")
plt.show()

## Image Prediction on Test Data

In [ ]:
y_pred = pre_model.predict(x_test_scaled)

In [ ]:
y_pred_test_label = [np.argmax(i) for i in y_pred]

## Actual and Predicted Images of Dogs

In [ ]:
y_pred_test_label[:10]  ## Predicted Images

In [ ]:
y_test[:10]  ## Actual Images

In [ ]:
plt.figure(figsize=(16,20))
for i in range(0,20):
    plt.subplot(5,4,i+1)
    plt.imshow(x_test[i])
    plt.title(f"Actual:{class_labels[y_test[i]]}\nPredicted:{class_labels[y_pred_test_label[i]]}")
    plt.axis("off")

# ResNet V2 Pretrained Model

### ResNet V-2 152 Layers 

In [ ]:
path = "https://tfhub.dev/google/imagenet/resnet_v2_152/classification/5"  # 224x224x3
resnet_pre_model = hub.KerasLayer(path,input_shape=(224,224,3),trainable=False)

In [ ]:
num_class = 4
resnet_model = Sequential()
resnet_model.add(resnet_pre_model)
resnet_model.add(Dense(units=num_class, activation="softmax"))

resnet_model.summary()

In [ ]:
resnet_model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
history_3 = resnet_model.fit(x_train_scaled, y_train ,
          batch_size=32, epochs=4, validation_data=(x_test_scaled, y_test))

## Model Evaluation on Test and Train Data

In [ ]:
loss, acc = resnet_model.evaluate(x_train_scaled,y_train)
print("Loss on Train Data:",loss)
print("Accuracy on Train Data:",acc)
print()
val_loss, val_acc = resnet_model.evaluate(x_test_scaled,y_test)
print("Loss on Test Data:",val_loss)
print("Accuracy on Test Data:",val_acc)

In [ ]:
y_pred = resnet_model.predict(x_test_scaled)
y_pred_test_label = [np.argmax(i) for i in y_pred]

print("Actual Label:",y_test[:10])
print("Predicted label:",y_pred_test_label[:10])

## Compare Actual and Predicted Images

In [ ]:
plt.figure(figsize=(16,20))
for i in range(0,20):
    plt.subplot(5,4,i+1)
    plt.imshow(x_test[i])
    plt.title(f"Actual:{class_labels[y_test[i]]}\nPredicted:{class_labels[y_pred_test_label[i]]}")
    plt.axis("off")

# EfficientNet Pre-trained Model

In [ ]:
path = "https://tfhub.dev/google/efficientnet/b0/classification/1" # 224x224x3
efficient_model = hub.KerasLayer(path,input_shape=(224,224,3),trainable=False)

num_class = 4
eff_model = Sequential()
eff_model.add(efficient_model)
eff_model.add(Dense(units=num_class, activation="softmax"))

eff_model.summary()

In [ ]:
eff_model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
history_4 = eff_model.fit(x_train_scaled , y_train ,
          batch_size=32, epochs=5, validation_data=(x_test_scaled,y_test))

In [ ]:
loss, acc = eff_model.evaluate(x_train_scaled,y_train)
print("Loss on Train Data:",loss)
print("Accuracy on Train Data:",acc)
print()
val_loss, val_acc = eff_model.evaluate(x_test_scaled,y_test)
print("Loss on Test Data:",val_loss)
print("Accuracy on Test Data:",val_acc)

In [ ]:
y_pred = eff_model.predict(x_test_scaled)
y_pred_test_label = [np.argmax(i) for i in y_pred]

print("Actual Label:",y_test[:10])
print("Predicted label:",y_pred_test_label[:10])

## Compare Actual and predicted Labels

In [ ]:
plt.figure(figsize=(16,20))
for i in range(0,20):
    plt.subplot(5,4,i+1)
    plt.imshow(x_test[i])
    plt.title(f"Actual:{class_labels[y_test[i]]}\nPredicted:{class_labels[y_pred_test_label[i]]}")
    plt.axis("off")